## Mission to Mars: Web Scraping Challenge

In [ ]:
#Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pymongo
import pandas as pd
import time
import os

In [ ]:
#Set Executable Path & Initialize Chrome Browser
executable_path = {"executable_path": (r"C:\Users\Mickey\anaconda3\Scripts\chromedriver.exe")}
browser = Browser("chrome", **executable_path)

In [ ]:
#Browse URL
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [ ]:
#Parse Results HTML with BeautifulSoup
html = browser.html
soup = bs(html, 'html.parser')

#### Mars News

In [ ]:
#Scrape Title
title = soup.find_all("div", class_="content_title")

news_title = title[1].text.strip()
news_title

In [ ]:
#Scrape Paragraph
news_paragraph = soup.find("div", class_="article_teaser_body").text
news_paragraph

#### Mars Images - Featured Image

In [ ]:
#Browse URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [ ]:
#Ask Splinter to Go to Site and Click Button with Class Name full_image
# <button class="full_image">Full Image</button>
full_image_button = browser.find_by_id("full_image")
full_image_button.click()

In [ ]:
#Find "More Info" Button and Click It
browser.is_element_present_by_text("more info", wait_time=1)
more_info_element = browser.find_link_by_partial_text("more info")
more_info_element.click()

In [ ]:
#Parse Results with BeautifulSoup 

html = browser.html
image_soup = bs(html, "html.parser")

img = image_soup.select_one("figure.lede a img")
img_url = img.get("src")

# combine with the base url
featured_img_url = f"https://www.jpl.nasa.gov{img_url}"

In [ ]:
#Print Image
featured_img_url

#### Mars Facts

In [ ]:
#Browse URL
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [ ]:
#Read Mars Facts using Pandas
facts_df = pd.read_html(url)[0]
facts_df.columns=["Description", "Value"]
facts_df

In [ ]:
#Return Results
facts_df.to_html('mars_facts.html', index=False)

#### Mars Hemispheres

In [ ]:
#Browse URL
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [ ]:
#Create an Empty List to store Result
hemi_img_urls = []

#Get a List of Hemispheres (products)
products = browser.find_by_css("a.product-item h3")

#Begin For Loop 

for item in range(len(products)):
    hemisphere = {}

    browser.find_by_css("a.product-item h3")[item].click()
    
    time.sleep(1)
    
    #Find Sample Image
    sample_element = browser.links.find_by_text("Sample").first
    hemisphere["img_url"] = sample_element["href"]

    #Get Hemisphere Title
    hemisphere["title"] = browser.find_by_css("h2.title").text

    #Add to List
    hemi_img_urls.append(hemisphere)

    #Navigate to Previous Pagee
    browser.back()

#Return Results
hemi_img_urls

In [ ]:
#Close Browser
browser.quit()